ST1:将json合并
将两个json合并并保存到另外一个文件夹

In [2]:
import json
import os

def merge_json_to_array(file1_path, file2_path, output_path):
    """将两个JSON文件合并为一个JSON数组"""
    try:
        # 读取第一个JSON文件
        with open(file1_path, 'r', encoding='utf-8') as f1:
            data1 = json.load(f1)
        
        # 读取第二个JSON文件
        with open(file2_path, 'r', encoding='utf-8') as f2:
            data2 = json.load(f2)
        
        # 合并为数组
        merged_data = [*data1, *data2]
        
        # 确保输出目录存在
        output_dir = os.path.dirname(output_path)
        os.makedirs(output_dir, exist_ok=True)
        
        # 保存合并后的JSON
        with open(output_path, 'w', encoding='utf-8') as out_file:
            json.dump(merged_data, out_file, ensure_ascii=False, indent=2)
        
        print(f"合并完成，结果已保存到: {output_path}")
        return True
    
    except Exception as e:
        print(f"错误: 合并JSON时出错 - {str(e)}")
        return False

# # 使用示例
# if __name__ == "__main__":
file1 = "/storage/human_psd/json/fp_v1.json"    # 替换为第一个JSON文件路径
file2 = "/storage/human_psd/json/tao_llz.json"   # 替换为第二个JSON文件路径
output = "/storage/human_psd/json/fp_llz_mer.json"  # 替换为输出路径

merge_json_to_array(file1, file2, output)

合并完成，结果已保存到: /storage/human_psd/json/fp_llz_mer.json


True

ST2：获取id
获取某个文件夹所有图片文件的id保存为json
例如：
0618_tao_llz_b0291fd717_2_80.png
0618_tao_llz_b0291fd717_2_23.png
id为0618_tao_llz_b0291fd717
0618_freepik_v1_0be3e0f79d_2_5.png
0618_freepik_v1_0be3e0f79d_2_6.png
id为0618_freepik_v1_0be3e0f79d


In [1]:
import os
import re
import json
import argparse
from tqdm import tqdm

def extract_id_from_filename(filename):
    """
    从图片文件名中提取ID
    
    参数:
    filename (str): 图片文件名
    
    返回:
    str: 提取到的ID，如果无法提取则返回None
    """
    # 正则表达式模式，用于匹配ID部分
    pattern = r'^(.+?)_\d+_\d+\.png$'
    match = re.match(pattern, filename)
    
    if match:
        return match.group(1)
    return None

def get_image_ids_from_folder(folder_path, extensions=['.png', '.jpg', '.jpeg']):
    """
    从文件夹中获取所有图片文件的ID
    
    参数:
    folder_path (str): 文件夹路径
    extensions (list): 支持的图片文件扩展名，默认为['.png', '.jpg', '.jpeg']
    
    返回:
    list: 提取到的ID列表
    """
    # 用于存储提取到的ID
    ids = []
    
    # 检查文件夹是否存在
    if not os.path.exists(folder_path):
        print(f"错误：文件夹 '{folder_path}' 不存在")
        return ids
    
    # 获取文件夹中的所有文件
    files = os.listdir(folder_path)
    
    # 使用tqdm显示进度条
    for filename in tqdm(files, desc="处理文件"):
        # 检查文件扩展名
        file_ext = os.path.splitext(filename)[1].lower()
        if file_ext in extensions:
            # 提取ID
            file_id = extract_id_from_filename(filename)
            if file_id:
                ids.append(file_id)
    
    return ids

def save_ids_to_json(ids, output_path):
    """
    将ID列表保存到JSON文件
    
    参数:
    ids (list): ID列表
    output_path (str): 输出JSON文件路径
    """
    # 确保输出文件夹存在
    output_dir = os.path.dirname(output_path)
    if output_dir and not os.path.exists(output_dir):
        os.makedirs(output_dir, exist_ok=True)
    
    # 保存ID列表到JSON文件
    try:
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(ids, f, ensure_ascii=False, indent=4)
        print(f"成功将 {len(ids)} 个ID保存到 '{output_path}'")
    except Exception as e:
        print(f"错误：无法保存JSON文件 '{output_path}': {str(e)}")

# def main():
#     """主函数，处理命令行参数"""
#     parser = argparse.ArgumentParser(description='从图片文件名中提取ID并保存到JSON文件')
#     parser.add_argument('-i', '--input', required=True, help='包含图片文件的文件夹路径')
#     parser.add_argument('-o', '--output', required=True, help='输出JSON文件路径')
#     parser.add_argument('-e', '--extensions', nargs='+', default=['.png', '.jpg', '.jpeg'], 
#                         help='支持的图片文件扩展名，默认为 .png .jpg .jpeg')
    
#     args = parser.parse_args()
extensions=['.png', '.jpg', '.jpeg']
input_path='/storage/human_psd/img_human_detected/filltered/fp_v2'
output_path='/storage/human_psd/json/fp_v2_id.json'

# 获取图片ID
ids = get_image_ids_from_folder(input_path, extensions)

# 保存到JSON文件
if ids:
    save_ids_to_json(ids, output_path)
else:
    print("警告：未找到任何有效的图片文件或无法提取ID")

# if __name__ == "__main__":
#     main()

处理文件: 100%|██████████| 254/254 [00:00<00:00, 259588.99it/s]

成功将 254 个ID保存到 '/storage/human_psd/json/fp_v2_id.json'


ST3:过滤json

In [2]:
merge_json_path='/storage/human_psd/json/fp_v2.json'
with open(merge_json_path, 'r', encoding='utf-8') as f1:
    data1 = json.load(f1)


In [3]:
from tqdm import tqdm
fillter_data=[]

for d in tqdm(data1):
    if d['id'] in ids:fillter_data.append(d)


100%|██████████| 528/528 [00:00<00:00, 82455.60it/s]


In [4]:
len(fillter_data)

254

In [5]:
filter_data_path='/storage/human_psd/filltered_json/fillter_fp_v2.json'

with open(filter_data_path, 'w', encoding='utf-8') as f:
    json.dump(fillter_data, f, ensure_ascii=False, indent=4)

In [6]:
# merge_json_path='/storage/human_psd/json/fp_llz_mer.json'
with open(filter_data_path, 'r', encoding='utf-8') as f1:
    test_m_data = json.load(f1)

len(test_m_data)

254